**TRATAMIENTO DE SEÑALES III**

*Segundo Mini Proyecto.*

*Modelos de Variable Latente para el Reconocimiento de Acciones
Motoras.*

*Luis Carlos Figueroa Benitez.*

*C.C: 1036649148*

**Universidad de Antioquia**

**Facultad de Ingeniería**

**2021-2**

In [1]:
# Montamos el drive.
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

#Ruta para la carpeta data.
root_path = '/content/gdrive/MyDrive/Señales_III/miniproyecto2/data/'

Mounted at /content/gdrive/


In [2]:
#from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import display
import seaborn as sns
from seaborn import pairplot
from sklearn.decomposition import PCA, KernelPCA
from ipywidgets import interact
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
@interact(input=['electromiografia', 'petalos'], n_components =(1,21,1), Flag_Space = ['Input','Latent','PCA','KPCA'],Kernel=['poly','rbf','linear','sigmoid','cosine'],Gamma=(1,20,1), Alpha=(0,0.9,0.1) )

def ExperimentoBayes(input, n_components, Flag_Space,Kernel,Gamma,Alpha):

  if input == 'electromiografia':
    data1 = pd.read_excel(root_path+'dataframe.xlsx')

    #Convertimos el dataframe a numpy
    data =data1.to_numpy()
    N,D= np.shape(data) 

    etiquetas = np.concatenate( (np.ones(189), 2*np.ones(189), 3*np.ones(189), 4*np.ones(189), 5*np.ones(189), 6*np.ones(189)) )
    etiquetas= etiquetas.astype(int)

  if input == 'petalos':
    iris = sns.load_dataset("iris")
    dataFrame_X = iris.drop(columns='species')
    data = dataFrame_X.to_numpy()

    N,D = np.shape(data)
    etiquetas = np.concatenate((np.ones(50),2*np.ones(50),3*np.ones(50)))
    etiquetas= etiquetas.astype(int)

  pca = PCA(n_components)  
  kernel_pca = KernelPCA(n_components, kernel=Kernel, gamma=Gamma, alpha=Alpha)

  # Load our dataset X
  X = data
  projectedZ = pca.fit_transform(X)
  BasisW = pca.components_
  print('Input Space:',data.shape)
  print('Latent Space:',projectedZ.shape)
  print(pca.explained_variance_)
  print(BasisW.shape)

  if Flag_Space == 'Input':
    X = data
    y = etiquetas
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

    fig, (input_x) = plt.subplots(ncols=1, figsize=(14, 4))
    input_x.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
    input_x.set_ylabel("Principal component #1")
    input_x.set_xlabel("Principal component #0")
    input_x.set_title("INPUT")

  if Flag_Space == 'Latent':
    X = projectedZ
    y = etiquetas
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

    fig, (latent_x) = plt.subplots(ncols=1, figsize=(14, 4))
    latent_x.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
    latent_x.set_ylabel("Principal component #1")
    latent_x.set_xlabel("Principal component #0")
    latent_x.set_title("LATENT")


  if Flag_Space == 'PCA':
    X = projectedZ
    y = etiquetas
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)
    pca.fit(X_train)
    X_test = pca.transform(X_test) # Primero ajusta con Xtrain y transforma a Xtest

    fig, (pca_proj_ax) = plt.subplots(ncols=1, figsize=(14, 4))
    pca_proj_ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
    pca_proj_ax.set_ylabel("Principal component #1")
    pca_proj_ax.set_xlabel("Principal component #0")
    pca_proj_ax.set_title("Projection of testing data\n using PCA")

  if Flag_Space == 'KPCA': 
    X = projectedZ
    y = etiquetas
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)
    #pca.fit(X_train)
    X_test = kernel_pca.fit(X_train).transform(X_test)
    fig, (kernel_pca_proj_ax) = plt.subplots(ncols=1, figsize=(14, 4))
    kernel_pca_proj_ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
    kernel_pca_proj_ax.set_ylabel("Principal component #1")
    kernel_pca_proj_ax.set_xlabel("Principal component #0")
    _ = kernel_pca_proj_ax.set_title("Projection of testing data\n using KernelPCA")

  # Instanciamos nuestro modelo Bayesiano
  clasificadorBayes = GaussianNB()
  clasificadorBayes.fit(X_train,y_train)
  # Realizar una prediccion
  y_pred = clasificadorBayes.predict(X_test)
  # Realizamo el análisis del error
  error = np.sum(np.abs(y_pred-y_test))
  Acierto = 1.- error/len(y_test)
  # Analizar algo que se conoce como una matriz de confusión
  CMAT = confusion_matrix(y_test,y_pred)
  porcentajeAcierto = np.sum(np.diag(CMAT)) / len(y_test)
  print('Acc(%): ',porcentajeAcierto)
  
  ax = ConfusionMatrixDisplay(CMAT)
  ax.plot()
  plt.show()

interactive(children=(Dropdown(description='input', options=('electromiografia', 'petalos'), value='electromio…

#Observaciones

- Nuestros datos a analizar es la señal electromiografica del anterior miniproyecto, ademas añadí los datos de los petalos enseñados en clase, con el fin de visualizar como se comportaba el algoritmo con menores datos y espacios latentes.

- al variar la dimension q en el espacio latente, se puede decir que estamos perdiendo informacion, sin embargo esta informacion que se pierde no es muy relevante para el analisis al reducir la dimension **q** muy poco, puesto que el accuracy varia poco; Al contrario al reducir **q** bastante, se pierde mucha informacion y nuestro accuracy ya empieza a ser muy bajo.

- los mejores accuracy se obtuvieron con los parametros **n_componntes** >= 10, **Flag_ space** =PCA Y KPCA, **Kernel**= lineal; con estos parametros el desempeño no es menor al 85%.
- En KPCA al variar los kernels, se logra identificar que el kernel lineal alcanza el mejor desemepeño, identificando el movimiento numero 5 como el de mayor varianza con respecto a los demas.

# Conclusiones

- PCA, estudia las variables de nuestro conjunto de datos,permite conservar la varianza maxima de los datos y a suvez reducir la dimension de los datos de entradas con menor componentes principales, estas son calculadas mediante un proceso iterativo donde se descompone las variables originales.
- KPCA, adicionalmente a PCA, Permite realizar el estudio de problemas no lineales convirtiendolos en problemas lineales para identificar de forma más sencilla patrones y clasificar los datos de entrada.